# 5. Andmete eeltöötlus


Sisukord:
* [Puuduvad andmed](#puudu)
* [Ülesanne 5.1](#5_1)
* [Sobimatut tüüpi (nominaal-, ordinaal-) andmete konverteerimine](#sobimatu)
* [Andmete skaleerimine: normaliseerimine ja standardiseerimine](#skaleeri)
* [Oluliste atribuutide väljavalimine](#atr)
* [Ülesanne 5.2](#5_2)

Vaata ka
* [SciKit-learn.preprocessing](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing)

Vt ka: Sebastian Raschka. Python Machine Learning. Ch. 4. Building Good Training Sets: Data Preprocessing.

<a id='puudu'></a>
## Puuduvad andmed

Reaalsete andmestike korral on sagedaseks probleemiks puuduvad andmed: küsitletav ei vasta mõnele küsimusele, ajaloolised andmed on puudulikud jne. Enamik andmekaeve meetodeid jäävad sellise sisendiga hätta. Tüüpilisteks viisideks puuduvate andmetega ümberkäimisel on vastavate ridade/veergude väljaviskamine või puuduvate väärtuste asendamine keskväärtusega.

In [157]:
import pandas as pd
from io import StringIO # StringIO klass esitab stringi faililaadse objektina

# Tekitame puuduvate väärtustega NaN (Not a Number) andmeraamistiku df, kasutades read_csv() meetodit
csv_data = """A,B,C,D
1.0, 2.0, 3.0, 4.0
5.0, 6.0,, 8.0
0.0, 11.0, 12.0,"""
df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,0.0,11.0,12.0,NaN


### Väljaviskamine

Kasutame `pandas DataFrame` meetodit [dropna()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html).

In [158]:
print("Puuduvad väärtused veeruti\n", df.isnull().sum())

Puuduvad väärtused veeruti
 A    0
B    0
C    1
D    1
dtype: int64


In [159]:
# Elimineerime puuduvate väärtustega read
df_clean1 = df.dropna()
df_clean1

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [160]:
 #Elimineerime puuduvate väärtustega veerud
df_clean2 = df.dropna(axis=1)
df_clean2

,A,B
0,1.0,2.0
1,5.0,6.0
2,0.0,11.0


In [161]:
# Viska välja ainult need read, kus kõik väärtused puuduvad
df.dropna(how="all")

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,0.0,11.0,12.0,NaN


In [162]:
# Jäta alles need read, kus on vähemalt 2 väärtust
df.dropna(thresh=2)

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,0.0,11.0,12.0,NaN


In [163]:
# Viska välja ainult need read, kus puuduvad veergude B või C väärtused
df.dropna(subset=['B', 'C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,0.0,11.0,12.0,NaN


### Asendamine

In [164]:
# Kasutame klassi Imputer, et asendada puuduvad väärtused (missing_values="NaN") keskmistega (strategy="mean")
#from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer
import numpy as np

imr = SimpleImputer(missing_values=np.nan, strategy="mean") 
imputed_data = imr.fit_transform(df)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [ 0. , 11. , 12. ,  6. ]])

In [165]:
# Asendatud imputed_data on np.array. 
# Teeme sellest pandas DataFrame objekti, millel on algsega samad veerunimed
imputed_df = pd.DataFrame(imputed_data, columns=df.columns)
imputed_df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,7.5,8.0
2,0.0,11.0,12.0,6.0


<a id='5_1'></a>
## Ülesanne 5.1

Eemaldada allolevast andmeraamistikust `df_ex` veerud, kus puuduvad kõik väärtused. Asendada ülejäänud  puuduvad väärtused keskmistega üle veergude.

<!-- Võtta aluseks [UCI Horse Colic (hobuste kõhuvalu) andmestik](https://archive.ics.uci.edu/ml/datasets/Horse+Colic), mille saab alla laadida https://archive.ics.uci.edu/ml/machine-learning-databases/horse-colic/horse-colic.data. -->

In [166]:
csv_ex = """A,B,C,D,E
1.0, 2.0, 3.0, 24.0,
15.0, 6.0,,8.0,
0.0, 11.0, 9.0,,"""
df_ex = pd.read_csv(StringIO(csv_ex))
df_ex

,A,B,C,D,E
0,1.0,2.0,3.0,24.0,NaN
1,15.0,6.0,NaN,8.0,NaN
2,0.0,11.0,9.0,NaN,NaN


<!--
"""
cols =                                  ["surgery?",
                                         "Age",
                                         "Hospital Nr",
                                         "rectal temp",
                                         "pulse",
                                         "respiratory rate",
                                         "temp of extermities",
                                         "peripheral pulse",
                                         "mucuos membranes",
                                         "capillary refill time",
                                         "pain",
                                         "peristalsis",
                                         "abdominal distension",
                                         "nasogastric tube"
                                         "nasogastric reflux",
                                         "nasogastric reflux PH",
                                         "rectal examination-feces",
                                         "abdomen",
                                         "packed cell volume",
                                         "total protein",
                                         "abdominocentesis appearance",
                                         "abdominocentesis total protein",
                                         "outcome",
                                         "surgical lesion?",
                                         "type of lesion 1",
                                         "type of lesion 2",
                                         "type of lesion 3",
                                         "cp_data"
                                         ]
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/horse-colic/horse-colic.data", 
                 delimiter=" ",  index_col=False)
#print(list(zip(df.columns, cols)))
#print(df.columns)
#print(cols)
df
"""
-->

In [167]:
df_clean2 = df.dropna(axis=1, how="all")
df_clean2

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,0.0,11.0,12.0,NaN


In [168]:
imr = SimpleImputer(missing_values=np.nan, strategy="mean") 
imputed_data = imr.fit_transform(df_clean2)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [ 0. , 11. , 12. ,  6. ]])

<a id='sobimatu'></a>
## Sobimatut tüüpi (nominaal-, ordinaal-) andmete konverteerimine 

* Nominaalandmed: kategooria, millel puudub järjestus. N: veregrupp A, B, AB, O
* Ordinaalandmed: järjestus on, aga astmete vahe ei pruugi olla üle skaala ühtne. N: meeldib väga, meeldib, neutraalne, ei meeldi, ei meeldi üldse.

Mõned meetodid (näiteks pertseptron) ootavad arvandmeid ja nende jaoks võib olla vaja nominaal- ja ordinaalatribuute teisendada.

In [169]:
patsiendi_df = pd.DataFrame({"veregrupp": ["A", "AB", "A", "O", "O"], 
                             "valu": ["puudub", "puudub", "kerge", "intensiivne", "kerge"]})
patsiendi_df                            

,veregrupp,valu
0,A,puudub
1,AB,puudub
2,A,kerge
3,O,intensiivne
4,O,kerge


Ordinaalatribuutide puhul on tihti piisav siltide teisendamine täisarvulisele skaalale. Siin sobib hästi kasutada nö. *mapping* sõnastikke, mille võti on väärtus vanal skaalal ja väärtus on väärtus uuel skaalal. `Dataframe[col].map(mapping)` meetod teisendab seejärel vastava veeru.

In [170]:
valu_map = {"puudub": 0, "kerge": 1, "intensiivne": 2}
patsiendi_df["valu"] = patsiendi_df["valu"].map(valu_map)
patsiendi_df

,veregrupp,valu
0,A,0
1,AB,0
2,A,1
3,O,2
4,O,1


Nominaalatribuutide puhul on selline lähenemine ebasobiv. Arvuline järjestus oleks petlik, sest sisuline järjestus puudub. Siin tekitatakse iga nominaalkategooria jaoks oma 0-1 atribuut (0-polnud see väärtus, 1-oli see väärtus). Seda võimaldab `sklearn` alammooduli `preprocessing`klass [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html), mis töötab arvandmete peal  või `pandas`mooduli funktsioon  [get_dummies(df)](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html), mis teisendab stringe sisaldavad veerud.

In [171]:
patsiendi_df = pd.get_dummies(patsiendi_df)
patsiendi_df

,valu,veregrupp_A,veregrupp_AB,veregrupp_O
0,0,1,0,0
1,0,0,1,0
2,1,1,0,0
3,2,0,0,1
4,1,0,0,1


Kui meil on vaja rakendada erinevatele veergudele erinevaid teisenduspoliitikaid, siis võib kasutada klassi [ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn.compose.ColumnTransformer), mis seab igale veerule vastavusse transformaatorobjekti (peab omama `fit()` ja `transform()` meetodeid. ColumnTransformeri initsialiseerimise sisendiks on (nime_str, transformaator, veeru_id või veeru_id list) kolmikute list. 

In [172]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

patsiendi_df2 = pd.DataFrame({"veregrupp": ["A", "AB", "A", "O", "O"], 
                             "valu": ["puudub", "puudub", "kerge", "intensiivne", "kerge"]})

# NB! OrdinalEncoder järjestab stringid tähestikulises järjekorras, sisuliselt nominaalskaala, mitte ordinaalskaala
ct = ColumnTransformer([("int_valu", OrdinalEncoder(), [1]),("binarize_veri", OneHotEncoder(), [0])])
ct.fit_transform(patsiendi_df2)

array([[2., 1., 0., 0.],
       [2., 0., 1., 0.],
       [1., 1., 0., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 1.]])

<a id='skaleeri'></a>
## Andmete skaleerimine: normaliseerimine ja standardiseerimine

Enamik masinõppe algoritme, va otsustuspuud, töötavad paremini kui kõik atribuudid on samal skaalal.
On kaks fundamentaalset lähenemist:
* **Normaliseerimine** \[0..1\] skaalale
* **Standardiseerimine** skaalale, kus keskväärtus on 0 ja standardhälve on 1.

Normaliseeritava veeru $x$  ja rea $i$ elemendi uus väärtus $x_{norm}^i$:
$$ x_{norm}^i = \frac{x^i - x_{min}}{x_{max} - x_{min}}$$
$x^i$: vana väärtus; $x_{max}, x_{min}$: veeru $x$ maksimaalne ja minimaalne väärtus.
Normaliseerimise eest vastutav klass on [sklearn.preprocessing.MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html).

Standardiseeritava veeru $x$  ja rea $i$ elemendi uus väärtus $x_{std}^i$:
$$ x_{std}^i = \frac{x^i - \mu_x}{\sigma_x}$$
$x^i$: vana väärtus; $\mu_x, \sigma_x$: veeru $x$ keskväärtus ja standardhälve.
Standardiseerimise eest vastutav klass on [sklearn.preprocessing.StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html).


In [173]:
from sklearn.preprocessing import MinMaxScaler

norm_data = MinMaxScaler().fit_transform(patsiendi_df)
print(norm_data)
patsiendi_df = pd.DataFrame(norm_data, columns=patsiendi_df.columns)
print("\n", patsiendi_df)

[[0.  1.  0.  0. ]
 [0.  0.  1.  0. ]
 [0.5 1.  0.  0. ]
 [1.  0.  0.  1. ]
 [0.5 0.  0.  1. ]]

    valu  veregrupp_A  veregrupp_AB  veregrupp_O
0   0.0          1.0           0.0          0.0
1   0.0          0.0           1.0          0.0
2   0.5          1.0           0.0          0.0
3   1.0          0.0           0.0          1.0
4   0.5          0.0           0.0          1.0


<a id='atr'></a>
## Oluliste atribuutide väljavalimine (*feature selection*)

Suure arvu atribuutide korral, eriti kui objektide arv on suhteliselt väike, on ülekohandamise (overfitting) probleem lihtne tekkima: objekti klassi saab ennustada atribuutide kombinatsiooni alusel, mis on unikaalne üksikobjektile. Selliselt treenitud klassifikaatorid annavad näiliselt häid tulemusi treeningandmetel, aga ennustustäpsus langeb tugevalt uute andmete korral. Samuti on sellised mudelid keerulised ja väikese üldistusjõuga. 

Seega on tihti kasulik atribuutide arvu vähendada. Siin on kaks fundamentaalset strateegiat:
* Uute atribuutide defineerimine olemasolevate atribuutide kombinatsioonina, nö dimensionaalsuse vähendamine/leidmine, mida vaatame järgmise nädala teema all. (*feature extraction*)
* Olemasolevatest atribuutidest sobivaimate väljavalimine. (*feature selection*)



Tihti on võimalik vähendada ennustamiseks kasutatavate atribuutide arvu klassifikaatorispetsiifiliste meetodite abil. Näiteks logistilise regressiooni korral suurendab nullkaalude arvu `penalty="l1"` (L1 regulariseerimine (lasso)) koos madala hinnaargumendiga `C`.

https://scikit-learn.org/stable/auto_examples/linear_model/plot_logistic_l1_l2_sparsity.html

Kaaluvektori **w** hinnafunktsioonile J(**w**) lisandub erinev regulariseerimise trahv:

L1: $$ \sum_{j=1..m} |w_j|$$

L2 (suurem trahv suurtele $w_j$ väärtustele): $$ \sum_{j=1..m} w_j^2$$




In [174]:
from sklearn.linear_model import LogisticRegression
from sklearn import datasets

iris = datasets.load_iris()
clf_general = LogisticRegression(penalty="l2", C=0.1, solver='liblinear')
clf_general.fit(iris.data, iris.target)
print("Tavalise logistilise regressiooni (penalty='l2' ja C=0.1) leitud kaalud:")
print(clf_general.coef_)

clf_sparse = LogisticRegression(penalty="l1", C=0.1, solver='liblinear')
clf_sparse.fit(iris.data, iris.target)
print("\n\nLogistilise regressiooni leitud kaalud, kui penalty='l1' ja C=0.1:")
print(clf_sparse.coef_)


Tavalise logistilise regressiooni (penalty='l2' ja C=0.1) leitud kaalud:
[[ 0.21310863  0.776912   -1.23987617 -0.55518357]
 [ 0.04423007 -0.62688956  0.29231476 -0.24855524]
 [-0.63387401 -0.5619517   0.94991857  0.78619837]]


Logistilise regressiooni leitud kaalud, kui penalty='l1' ja C=0.1:
[[ 0.          1.12130038 -1.34325149  0.        ]
 [ 0.         -0.38712006  0.12333704  0.        ]
 [-0.98736083  0.          1.27638939  0.        ]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Klass [SelectKBest](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html) võimaldab valida välja $k$ parimat atribuuti teatud hinnangufunktsiooni järgi. Hinnangufunktsioon võib olla statistiline ($\chi^2$, ANOVA F statistik,...) või näiteks valepositiivsete tulemuste arv atribuudi järgi ennustades.

In [175]:
from sklearn.feature_selection import SelectKBest, chi2

# Tekitame SelectKBest transformaatori,
# treenime ja saame vastuse fit_transform() meetodi abil
k_best = SelectKBest(chi2, k=2)
X_new = k_best.fit_transform(iris.data, iris.target)
print(k_best.scores_)
X_new[:10]

[ 10.81782088   3.7107283  116.31261309  67.0483602 ]


array([[1.4, 0.2],
       [1.4, 0.2],
       [1.3, 0.2],
       [1.5, 0.2],
       [1.4, 0.2],
       [1.7, 0.4],
       [1.4, 0.3],
       [1.5, 0.2],
       [1.4, 0.2],
       [1.5, 0.1]])

Atribuutide väljavalimiseks saame kasutada ka juhusliku metsa [RandomForest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) ansamblimeetodi omadust `feature_importances_`.

In [176]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)
forest.fit(iris.data, iris.target)
print(forest.feature_importances_)

[0.09793568 0.02339509 0.43979316 0.43887607]


<a id='5_2'></a>
## Ülesanne 5.2

a) Allpool tekitatakse veinide andmetabel $X$ ja klasside vektor $y$.

* Normaliseerida see tabel [0..1] skaalale
* Leida logistilise regressiooni mudel, kus nullist erinevate kaalude arv on L1 regulariseerimise abil minimeeritud.
* Hinnata atribuutide olulisust juhusliku metsa meetodil.

In [177]:
wine = datasets.load_wine()
wineData = MinMaxScaler().fit_transform(wine.data)
X = pd.DataFrame(wine.data, columns=wine.feature_names)
y = wine.target
X

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


In [178]:
clf_sparse = LogisticRegression(penalty="l1", solver='liblinear', C=0.1)
clf_sparse.fit(wineData, wine.target)
print("\n\nLogistilise regressiooni leitud kaalud, kui penalty='l1'")
print(clf_sparse.coef_)



Logistilise regressiooni leitud kaalud, kui penalty='l1'
[[ 0.          0.          0.         -0.93737669  0.          0.
   0.         -0.55699467  0.          0.          0.          0.
   0.58656388]
 [-0.9488223   0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.         -2.32737778
   0.        ]]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [179]:
forest = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)
forest.fit(wineData, wine.target)
print(forest.feature_importances_)

[0.12255305 0.03056653 0.01365851 0.02673632 0.03067907 0.0529636
 0.15863861 0.01091742 0.02130066 0.15957213 0.08219627 0.11728454
 0.1729333 ]


b) Tutvuda ja laadida alla UCI [Teaching Assistant Evaluation](https://archive.ics.uci.edu/ml/datasets/Teaching+Assistant+Evaluation) andmestik ([tae.data](https://archive.ics.uci.edu/ml/machine-learning-databases/tae/tae.data)).


Leida dokumentatsiooni alusel nominaalatribuudid (categorical) ja konverteerida need kahendkujule kasutades funktsiooni [pd.get_dummies(dataframe, columns=[col1, col2,..])](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html), kus `columns` on list veerunimedest, mida tuleb kahendkujule teisendada.




In [180]:
# Laadimise näide:
tae_df=pd.read_csv("tae.data",
                   header=None,
                   names=["Native_Eng", "Instructor", "Course", "Semester_type", "Class_size", "Class"])
tae_df

,Native_Eng,Instructor,Course,Semester_type,Class_size,Class
0,1,23,3,1,19,3
1,2,15,3,1,17,3
2,1,23,3,2,49,3
3,1,5,2,2,33,3
4,2,7,11,2,55,3
...,...,...,...,...,...,...
146,2,3,2,2,26,1
147,2,10,3,2,12,1
148,1,18,7,2,48,1
149,2,22,1,2,51,1


In [181]:
tae_df = pd.get_dummies(tae_df, columns=["Class", "Instructor", "Course"])
tae_df

,Native_Eng,Semester_type,Class_size,Class_1,Class_2,Class_3,Instructor_1,Instructor_2,Instructor_3,Instructor_4,...,Course_17,Course_18,Course_19,Course_20,Course_21,Course_22,Course_23,Course_24,Course_25,Course_26
0,1,1,19,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,17,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,49,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2,33,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,2,55,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,2,2,26,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
147,2,2,12,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
148,1,2,48,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
149,2,2,51,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
